## Search Volumes

In [31]:
# imports
import pandas as pd
import os
from dotenv import load_dotenv
from serpapi import GoogleSearch
from datetime import date


Importing search terms csv as python dataframe.

In [32]:
# Getting serpapi key
load_dotenv()
api_key = os.getenv("SERPAI_KEY")

# Read the CSV file, skipping the first two rows
search_terms_df = pd.read_csv("Search_terms.csv", skiprows=2)

C:\Users\emily\AppData\Local\Temp\ipykernel_205024\1943439807.py:6: DtypeWarning: Columns (8,14) have mixed types. Specify dtype option on import or set low_memory=False.
  search_terms_df = pd.read_csv("Search_terms.csv", skiprows=2)


In [33]:
search_terms_df.head()

,Search term,Match type,Added/Excluded,Campaign,Ad group,Currency code,Avg. CPM,Impr.,Interactions,Interaction rate,Avg. cost,Cost,Campaign type,Conv. rate,Conversions,Cost / conv.
0,mental health support line,Exact match (close variant),NaN,Nebo - Helmsley Grant - Nonbrand - Crisis Support,Helpline / Hotline,USD,280.00,40,5,12.50%,2.24,11.20,Search,180.00%,9.0,1.24
1,depression support group,Broad match,NaN,Nebo - Helmsley Grant - Nonbrand - Crisis Support,Helpline / Hotline,USD,187.06,17,1,5.88%,3.18,3.18,Search,100.00%,1.0,3.18
2,teen help hotline,Broad match,NaN,Nebo - Helmsley Grant - Nonbrand - Crisis Support,Suicide Prevention,USD,456.25,16,4,25.00%,1.83,7.30,Search,100.00%,4.0,1.83
3,family acceptance project lds,Broad match,NaN,Nebo - Helmsley Grant - Nonbrand - Crisis Support,Helpline / Hotline,USD,840.00,1,1,100.00%,0.84,0.84,Search,100.00%,1.0,0.84
4,hope line,Broad match,NaN,Nebo - Helmsley Grant - Nonbrand - Crisis Support,Suicide Prevention,USD,605.00,6,1,16.67%,3.63,3.63,Search,200.00%,2.0,1.82


Functions to remove duplciates and return percent change. Percent change is based on "interest_over_time" values from Google Search SerpAPI rather than search volume, search volume ain't publicly released (I think (gurt: sybau 🥀))

In [35]:
from serpapi import GoogleSearch

params = {
  "engine": "google",
  "q": "Coffee",
  "location": "Austin, Texas, United States",
  "google_domain": "google.com",
  "gl": "us",
  "hl": "en",
  "api_key": api_key
}

search = GoogleSearch(params)
results = search.get_dict()
results

{'search_metadata': {'id': '6812a9c29d42649d8923c38c',
  'status': 'Success',
  'json_endpoint': 'https://serpapi.com/searches/8fa8058388dba07e/6812a9c29d42649d8923c38c.json',
  'created_at': '2025-04-30 22:52:50 UTC',
  'processed_at': '2025-04-30 22:52:50 UTC',
  'google_url': 'https://www.google.com/search?q=Coffee&oq=Coffee&uule=w+CAIQICIaQXVzdGluLFRleGFzLFVuaXRlZCBTdGF0ZXM&hl=en&gl=us&sourceid=chrome&ie=UTF-8',
  'raw_html_file': 'https://serpapi.com/searches/8fa8058388dba07e/6812a9c29d42649d8923c38c.html',
  'total_time_taken': 2.39},
 'search_parameters': {'engine': 'google',
  'q': 'Coffee',
  'location_requested': 'Austin, Texas, United States',
  'location_used': 'Austin,Texas,United States',
  'google_domain': 'google.com',
  'hl': 'en',
  'gl': 'us',
  'device': 'desktop'},
 'search_information': {'query_displayed': 'Coffee',
  'total_results': 4430000000,
  'time_taken_displayed': 0.36,
  'organic_results_state': 'Results for exact spelling',
  'results_for': 'Austin, TX'}

In [26]:
def remove_duplicates(df):
    return df.drop_duplicates(subset=[df.columns[0]])

In [ ]:
def percent_change_in_search_volume(keyword, start_date, end_date):
    percent_change = 0
    
    # Format the date range for SerpAPI
    date_range = f"{start_date.strftime('%Y-%m-%d')} {end_date.strftime('%Y-%m-%d')}"

    params = {
        "engine": "google_trends",
        "q": keyword,               # Search word we're looking at
        "geo": "US",             # Restrict results to the United States
        "date": date_range,    # Time Range
        "api_key": api_key
    }

    
    search = GoogleSearch(params)
    results = search.get_dict()

    interest_data = results.get("interest_over_time", [])
    start_value = interest_data[0]["value"]
    end_value = interest_data[-1]["value"]
    percent_change = (end_value - start_value)/start_value * 100 if start_value != 0 else None

    return percent_change

In [21]:
start_date = date(2024, 1, 1)
end_date = date(2024, 12, 31)
search_terms_percent_change_list = []

search_terms_df = remove_duplicates(search_terms_df)
search_terms = search_terms_df.iloc[:, 0].tolist()

#returns an array of the percent changes of each keyword
for term in search_terms:
    search_terms_percent_change_list.append(percent_change_in_search_volume(term, start_date, end_date))

search_terms_df = search_terms_df.with_columns('Growth Percentage', search_terms_percent_change)
sorted_search_terms_df = search_terms_df.sort('Growth Percentage', descending = True)
top_10_keywords = sorted_search_terms_df["Search term"].head(10).tolist()

top_10_keywords


[]


IndexError: list index out of range

interest_over_time in SerpApi’s Google Trends engine refers to how popular a search term is over a specified period, Google doesn't publicly release information about search volume. 

However, there is publicly released information about Google Ad search volume. To be more accurate, we can try to benchmark some of our search words against that in the future.